In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import tensorflow_datasets as tfds
from collections import Counter
import scipy.ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import gc
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tarfile
import urllib.request
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F
from scipy.stats import gaussian_kde
from numba import jit
from torchvision.datasets import INaturalist
from tqdm import tqdm  # 導入進度條庫
import json
import torch.nn as nn

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
torch.backends.cudnn.benchmark = True
limit = 1000
num_samples = 1  # Monte-Carlo Dropout 取樣次數
dropout_rate = 0.0

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

In [ ]:
# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

class MCDropoutResNet(nn.Module):
    def __init__(self, dropout_p):
        super().__init__()
        # **載入與 checkpoint 匹配的 ResNet50**
        self.resnet = models.resnet50(weights=None)

        # **修改 FC 層輸出至 14 類別**
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 14)  # 確保輸出為 14 類別

        # **Dropout 參數**
        self.dropout_p = dropout_p

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = nn.Dropout(p=self.dropout_p)(x)  # 在 layer1 之後插入 Dropout

        x = self.resnet.layer2(x)
        x = nn.Dropout(p=self.dropout_p)(x)  # 在 layer2 之後插入 Dropout

        x = self.resnet.layer3(x)
        x = nn.Dropout(p=self.dropout_p)(x)  # 在 layer3 之後插入 Dropout

        x = self.resnet.layer4(x)
        x = nn.Dropout(p=self.dropout_p)(x)  # 在 layer4 之後插入 Dropout

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.resnet.fc(x)

        return x
    
    def enable_dropout(self):
        """啟用 Dropout，使其在推論時仍作用"""
        for m in self.modules():
            if isinstance(m, nn.Dropout):
                m.train()  # 在推論時啟用 Dropout

# **初始化模型**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MCDropoutResNet(dropout_p=dropout_rate).to(device)

# **載入權重**
checkpoint_path = "resnet50_epoch_50.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)

# **為所有鍵值新增 "resnet." 前綴**
new_checkpoint = {f"resnet.{k}": v for k, v in checkpoint.items()}  # 加上前綴
model.load_state_dict(new_checkpoint)

model.eval()  # 切換至推論模式
if dropout_rate > 0.0:
    print("enable dropout")
    model.enable_dropout()  # 啟用 MC Dropout

In [ ]:
# 得到了bzq 正確的函數，拿來做imagenet 正確的預測

def single_data_bzq_mask_preprocessing_imagenet(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data.cpu())
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x, :] *= magnification
    return new_data


#print(random_num_for_bzq_mask_imagenet)

def single_data_bzq_mask_preprocessing_imagenet_random_global(original_data, start_x, start_y, len_x, len_y, random_num_for_bzq_mask_imagenet):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data.cpu().numpy())
    new_data = new_data.squeeze(0)
    random_num_for_bzq_mask_imagenet = random_num_for_bzq_mask_imagenet[:, :len_y, :len_x] 
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] = random_num_for_bzq_mask_imagenet
    return new_data


bzq = []
correct_predictions_imagenet = []
incorrect_predictions_imagenet = []
bzq_imagenet = []

#bzq = 0的時候，提取mask之下的softmax
correct_predictions_bzq_zero_softmax_mean = []
correct_predictions_bzq_zero_softmax_std = []
incorrect_predictions_bzq_zero_softmax_mean = []
incorrect_predictions_bzq_zero_softmax_std = []


In [ ]:
class DummyDataset(Dataset):
    def __init__(self, limit):
        self.limit = limit

    def __len__(self):
        return self.limit

    def __getitem__(self, idx):
        img = torch.randn(3, 224, 224)
        label = torch.tensor(-1)    
        return img, label

test_dataset = DummyDataset(limit)

# 創建 DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)


In [ ]:
conf_vanilla = []
# 預測並顯示分數

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device).float()
        labels = labels.to(device)
        outputs = model(images)

        # 計算Softmax並獲取最大值
        softmax_probs = F.softmax(outputs, dim=1)  # Softmax值
        max_probs, predicted = torch.max(softmax_probs, 1)

        conf_vanilla.extend(max_probs.cpu().numpy()) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(model)
        #print(labels, predicted)
        
conf_vanilla = np.array(conf_vanilla)
accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
original_predictions_imagenet = []

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device).float()
        batch_predictions_imagenet = model(batch_data).cpu().numpy()
        original_predictions_imagenet.append(batch_predictions_imagenet)

original_predictions_imagenet = np.vstack(original_predictions_imagenet)

# 使用正確的方式來訪問 test_dataset 中的標籤
test_labels = [test_dataset[i][1] for i in range(len(test_dataset))]

predicted_labels = np.argmax(original_predictions_imagenet, axis=1)
correct_indices = np.where(predicted_labels == np.array(test_labels))[0]
incorrect_indices = np.where(predicted_labels != np.array(test_labels))[0]

correct_predictions_imagenet.extend(correct_indices.tolist())
incorrect_predictions_imagenet.extend(incorrect_indices.tolist())

print(f"{len(correct_predictions_imagenet)}, {len(incorrect_predictions_imagenet)}")

In [ ]:
import torch
import numpy as np
from collections import Counter
import torch.nn.functional as F

random_num_for_bzq_mask_imagenet = np.random.randint(0, 256, (3, 0, 0)).astype(np.float32) / 255.0

def compute_entropy(softmax_predictions):
    # 為了數值穩定，加上 ε 避免 log(0)
    eps = 1e-8
    
    log_probs = torch.log(softmax_predictions + eps)
    entropy = -torch.sum(softmax_predictions * log_probs)
    return entropy

def process_batch(batch_data, batch_label, device, model, bzq, mc_samples):
    if len(batch_data) == 0:
        return bzq, np.array([])

    mc_predictions = []
    
    for _ in range(mc_samples):  # **進行多次 forward**
        predictions = model(batch_data.float())
        mc_predictions.append(predictions.unsqueeze(0))  # **儲存 MC Dropout 結果**

    # **堆疊所有 MC Dropout 結果**
    mc_predictions = torch.cat(mc_predictions, dim=0)  # (mc_samples, batch_size, num_classes)

    # **計算 MC Dropout 之後的平均**
    mean_predictions = mc_predictions.mean(dim=0)  # 均值 (batch_size, num_classes)

    max_bzq_indices = torch.argmax(mean_predictions, dim=1).cpu().numpy()

    # **計算 softmax**
    softmax_predictions = F.softmax(mean_predictions, dim=1)

    #print(softmax_predictions.shape)
    entropy = compute_entropy(softmax_predictions)

    labels = batch_label

    counter = Counter(max_bzq_indices)
    most_common_num, most_common_count = counter.most_common(1)[0]
    
    temp = softmax_predictions[:, most_common_num].cpu().numpy()

    return temp, entropy.cpu()

entropies = []
with torch.no_grad():
    for batch_data, batch_label in test_loader:
        batch_data = batch_data.to(device)  # 將批次影像搬到指定裝置
        batch_label = batch_label.to(device)
        
        if len(batch_data) > 0:
            bzq, entropy = process_batch(
                batch_data, batch_label, device, model, bzq, num_samples
            )
            bzq_imagenet.append(bzq)
            entropies.append(entropy)

            # 影像預處理
            original_data = single_data_bzq_mask_preprocessing_imagenet(batch_data, 0, 0, 0, 0, 0)
            original_prediction = model(torch.tensor(original_data.reshape(-1, 3, 224, 224)).float().to(device))
            max_original_index = torch.argmax(original_prediction).item()


In [ ]:
# 計算熵的分佈（分桶）
bins = np.linspace(min(entropies), max(entropies), 15) 
hist, bin_edges = np.histogram(entropies, bins=bins)  # 計算直方圖數據
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # 計算每個 bin 的中心點
print(np.median(entropies), np.std(entropies))
# 繪製折線圖
plt.figure(figsize=(10, 5))
plt.plot(bin_centers, hist, linestyle='-', marker='o')

# 設定標題與軸標籤
plt.title("Entropy Distribution")
plt.xlabel("Entropy")
plt.ylabel("# of Examples")
plt.legend()
plt.grid(True)
plt.show()